# Machine Learning Basics

## Objectives

- Build binary classification models that predict activity/inactivity of small molecules against human aromatase using supervised learning methods.
- Evaluate the performance of the developed models using performance measures.

## 1. Import bioactivity data from PubChem

In this notebook, we will develop a prediction model for small molecule's activity against human aromatase (https://pubchem.ncbi.nlm.nih.gov/protein/EAW77416), which is encoded by the CYP19A1 gene (https://pubchem.ncbi.nlm.nih.gov/gene/1588). The model will predict the activity of a molecule based on the structure of the molecule (represented with molecular fingerprints).

For model development, we will use the Tox21 bioassay data for human aromatase, archived in PubChem (https://pubchem.ncbi.nlm.nih.gov/bioassay/743139).  The bioactivity data presented on this page can be downloaded by clicking the "Download" button available on this page and then read the data into a data frame.  Alternatively, you can directly load the data into a data frame as shown in the cell below.

In [ ]:
import pandas as pd
import numpy as np

url = 'https://pubchem.ncbi.nlm.nih.gov/assay/pcget.cgi?query=download&record_type=datatable&actvty=all&response_type=save&aid=743139'
df_raw = pd.read_csv(url)

In [ ]:
df_raw.head(7)

__Note:__ Lines 0-2 provide the descriptions for each column (data type, descriptions, units, etc).  These rows need be removed.

In [ ]:
df_raw = df_raw[3:]
df_raw.head(5)

The column names in this data frame contain white spaces and special characters.  For simplicity, let's rename the columns (no spaces or special characters except for the "_" character.)

In [ ]:
df_raw.columns

In [ ]:
col_names_map = {'PUBCHEM_RESULT_TAG' : 'pc_result_tag', 
                 'PUBCHEM_SID' : 'sid', 
                 'PUBCHEM_CID' : 'cid',
                 'PUBCHEM_ACTIVITY_OUTCOME' : 'activity_outcome', 
                 'PUBCHEM_ACTIVITY_SCORE' : 'activity_score',
                 'PUBCHEM_ACTIVITY_URL' : 'activity_url', 
                 'PUBCHEM_ASSAYDATA_COMMENT' : 'assay_data_comment', 
                 'Activity Summary' : 'activity_summary',
                 'Antagonist Activity' : 'antagonist_activity', 
                 'Antagonist Potency (uM)' : 'antagonist_potency', 
                 'Antagonist Efficacy (%)' : 'antagonist_efficacy',
                 'Viability Activity' : 'viability_activity', 
                 'Viability Potency (uM)' : 'viability_potency',
                 'Viability Efficacy (%)' : 'viability_efficacy', 
                 'Sample Source' : 'sample_source' }

In [ ]:
df_raw = df_raw.rename(columns = col_names_map)
df_raw.columns

## 2. Check the number of compounds for each activity group

First, we need to understand what our data look like.  Especially, we are interested in the activity class of the tested compounds because we are developing a model that classifies small molecules according to their activities against the target.  This information is available in the "**activity_outcome**" and "**activity_summary**" columns.

In [ ]:
df_raw.groupby(['activity_outcome']).count()

Based on the data in the **activity_outcome** column, there are 379 actives, 7562 inactives, and 2545 inconclusives.

In [ ]:
df_raw.groupby(['activity_outcome','activity_summary']).count()

Now, we can see that, in the **activity_summary** column, the inconclusive compounds are further classified into subclasses, which include:

- **active agonist**
- inconclusive
- inconclusive agonist
- inconclusive antagonist
- inconclusive agonist (cytotoxic)
- inconclusive antagonist (cytotoxic)



As implied in the title of this assay record (https://pubchem.ncbi.nlm.nih.gov/bioassay/743139), this assay aims to identify **aromatase inhibitors**.  Therefore, all **active antagonists** (in the activity summary column) were declared to be **active** compounds (in the activity outcome column).

On the other hand, the assay also identified 612 **active agonists** (in the activity summary column), and they are declared to be **inconclusive** (in the activity outcome column).

With that said, "inactive" compounds in this assay means those which are neither active agonists nor active antagonist.

It is important to understand that the criteria used for determining whether a compound is active or not in a given assay are selected by the data source who submitted that assay data to PubChem.  For the purpose of this assignment (which aims to develop a binary classifier that tells if a molecule is active or inactive against the target), we should clarify what we mean by "active" and "inactive".

- **active** : any compounds that can change (either increase or decrease) the activity of the target.  This is equivalent to either **active antagonists** or **active agonists** in the activity summary column.
- **inactive** : any compounds that do not change the activity of the target.  This is equivalent to **inactive** compounds in the activity summary column.

## 3. Select active/inactive compounds for model building

Now we want to select only the active and inactive compounds from the data frame (that is, active agonists, active antagonists, and inactives based on the "activity summary" column).

In [ ]:
df = df_raw[ (df_raw['activity_summary'] == 'active agonist' ) | 
             (df_raw['activity_summary'] == 'active antagonist' ) |
             (df_raw['activity_summary'] == 'inactive' ) ]

len(df)

In [ ]:
print(len(df['sid'].unique()))
print(len(df['cid'].unique()))

Note that the number of CIDs is not the same as the number of SIDs.  There are two important potential reasons for this observation.  

First, not all substances (SIDs) in PubChem have associated compounds (CIDs) because some substances failed during structure standardization.  \[Remember that, in PubChem, substances are depositor-provided structures and compounds are unique structures extracted from substances through structure standardization.]  Because our model will use structural information of molecules to predict their bioactivity, we need to remove substances without associated CIDs (i.e., no standardized structures).

Second, some compounds are associated with more than one substances.  In the context of this assay, it means that a compound may be tested multiple times in different samples (which are designated as different substances).  It is not uncommon that different samples of the same chemical may result in conflicting activities (e.g., active agonist in one sample but inactive in another sample).  In this practice, we remove such compounds with conflicting activities.

## 3-(1) Drop substances without associated CIDs.

First, check if there are subtances without associated CIDs.

In [ ]:
df.isna().sum()

There are 138 records whose "cid" column is NULL, and we want to remove those records.

In [ ]:
df = df.dropna( subset=['cid'] )
len(df)

In [ ]:
print(len(df['sid'].unique()))
print(len(df['cid'].unique()))

In [ ]:
df.isna().sum()   # Check if the NULL values disappeared in the "cid" column

## 3-(2) Remove CIDs with conflicting activities

Now identify compounds with conflicting activities and remove them.

In [ ]:
cid_conflict = []
idx_conflict = []

for mycid in df['cid'].unique() :
    
    outcomes = df[ df.cid == mycid ].activity_summary.unique()
    
    if len(outcomes) > 1 :
        
        idx_tmp = df.index[ df.cid == mycid ].tolist()
        idx_conflict.extend(idx_tmp)
        cid_conflict.append(mycid)

print("#", len(cid_conflict), "CIDs with conflicting activities [associated with", len(idx_conflict), "rows (SIDs).]")

In [ ]:
df.loc[idx_conflict,:].head(10)

In [ ]:
df = df.drop(idx_conflict)

In [ ]:
df.groupby('activity_summary').count()

In [ ]:
print(len(df['sid'].unique()))
print(len(df['cid'].unique()))

## 3-(3) Remove redundant data

The above code cells \[in 3-(2)] do not remove compounds tested multiple times if the testing results are consistent [e.g., active agonist in all samples (substances)].  The rows corresponding to these compounds are redundant, so we want remove them except for only one row for each compound.

In [ ]:
df = df.drop_duplicates(subset='cid')  # remove duplicate rows except for the first occurring row.
print(len(df['sid'].unique()))
print(len(df['cid'].unique()))

## 3-(4) Adding "numeric" activity classes

In general, the inputs and outputs to machine learning algorithms need to have numerical forms.   
In this practice, the input (molecular structure) will be represented with binary fingerprints, which already have numerical forms (0 or 1).  However, the output (activity) is currently in a string format (e.g., 'active agonist', 'active antagonist').  Therefore, we want to add an additional, 'activity' column, which contains numeric codes representing the active and inactive compounds:
- 1 for actives (either active agonists or active antagonists)
- 0 for inactives

Note that we are merging the two classes "active agonist" and "active antagonist", because we are going to build a binary classifer that distinguish actives from inactives.

In [ ]:
df['activity'] = [ 0 if x == 'inactive' else 1 for x in df['activity_summary'] ]

Check if the new column 'activity' is added to (the end of) the data frame.

In [ ]:
df.head(3)

Double-check the count of active/inactive compounds.

In [ ]:
df.groupby('activity_summary').count()

In [ ]:
df.groupby('activity').count()

## 3-(5) Create a smaller data frame that only contains CIDs and activities.

Let's create a smaller data frame that only contains CIDs and activities.  This data frame will be merged with a data frame containing molecular fingerprint information.

In [ ]:
df_activity = df[['cid','activity']]

In [ ]:
df_activity.head(5)

## 4. Download structure information for each compound from PubChem

Now we want to get structure information of the compounds from PubChem (in isomeric SMILES).

In [ ]:
cids = df.cid.astype(int).tolist()

In [ ]:
chunk_size = 200
num_cids = len(cids)

if num_cids % chunk_size == 0 :
    num_chunks = int( num_cids / chunk_size )
else :
    num_chunks = int( num_cids / chunk_size ) + 1

print("# CIDs = ", num_cids)
print("# CID Chunks = ", num_chunks, "(chunked by ", chunk_size, ")")

In [ ]:
import time
import requests
from io import StringIO

df_smiles = pd.DataFrame()
list_dfs = []  # temporary list of data frames

for i in range(0, num_chunks) :
    
    idx1 = chunk_size * i
    idx2 = chunk_size * (i + 1)
    cidstr = ",".join( str(x) for x in cids[idx1:idx2] )

    url = ('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/' + cidstr + '/property/IsomericSMILES/TXT')
    res = requests.get(url)
    data = pd.read_csv( StringIO(res.text), header=None, names=['smiles'] )
    list_dfs.append(data)
    
    time.sleep(0.2)
    
    if ( i % 5 == 0 ) :
        print("Processing Chunk ", i)   

#    if ( i == 2 ) : break  #- for debugging

df_smiles = pd.concat(list_dfs,ignore_index=True)
df_smiles[ 'cid' ] = cids
df_smiles.head(5)

In [ ]:
len(df_smiles)

In [ ]:
df_smiles = df_smiles[['cid','smiles']]
df_smiles.head(5)

## 5. Generate MACCS keys from SMILES.

In [ ]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys

In [ ]:
fps=dict()

for idx, row in df_smiles.iterrows() :
    
    mol = Chem.MolFromSmiles(row.smiles)
    
    if mol == None :
        print("Can't generate MOL object:", "CID", row.cid, row.smiles)
    else:
        fps[row.cid] = [row.cid] + list(MACCSkeys.GenMACCSKeys(mol).ToBitString())

In [ ]:
# Generate column names
fpbitnames = []

fpbitnames.append('cid')

for i in range(0,167):   # from MACCS000 to MACCS166
    fpbitnames.append( "maccs" + str(i).zfill(3) )

df_fps = pd.DataFrame.from_dict(fps, orient='index', columns=fpbitnames)

In [ ]:
df_fps.head(5)

## 6. Merge activity data and fingerprint information

In [ ]:
df_activity.head(3)

In [ ]:
df_fps.head(3)

In [ ]:
df_data = df_activity.join(df_fps.set_index('cid'), on='cid')

In Section 5, there were two CIDs for which the MACCS keys could not be generated.  They need to be removed from **df_data**.

In [ ]:
df_data[df_data.isna().any(axis=1)]

In [ ]:
df_data = df_data.dropna()
len(df_data)

Save df_data in CSV for future use.

In [ ]:
df_data.to_csv('df_data.csv')

## 7. Preparation for model building

## 7-(1) Loading the data into X and y.

In [ ]:
df_data.head(3)

In [ ]:
X = df_data.iloc[:,2:]
y = df_data['activity'].values

In [ ]:
X.head(3)

In [ ]:
print(len(y))    # Number of all compounds
y.sum()          # Number of actives

## 7-(2) Remove zero-variance features

Some features in X are not helpful in distinguishing actives from inactives, because they are set ON for all compounds or OFF for all compounds.  Such features need to be removed because they would consume more computational resources without improving the model.

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
X.shape  #- Before removal

In [ ]:
sel = VarianceThreshold()
X=sel.fit_transform(X)
X.shape  #- After removal

In this case, four features had zero variances.  Note that one of them is the first bit (maccs000) of the MACCS keys, which is added as a "dummy" to name each of bits 1~166 as maccs001, maccs002, ... maccs166.  

## 7-(3) Train-Test-Split (a 9:1 ratio)

Now split the data set into a training set (90%) and test set (10%).  The training set will be used to train the model.  The developed model will be tested against the test set.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, shuffle=True, random_state=3100, stratify=y, test_size=0.1)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_train.sum(), y_test.sum())

## 7-(4) Balance the training set through downsampling

Check the dimension of the training data set.

In [ ]:
print(len(y_train))
print(len(X_train))
print(len(X_train[0]))

Check the number of actives and inactives compound.

In [ ]:
print("# inactives : ", len(y_train) - y_train.sum())
print("# actives   : ", y_train.sum())

The data set is highly imbalanced \[the inactive to active ratio is 8.16 (=5448 / 668)].  To address this issue, let's downsample the majority class (inactive compounds) to balance the data set.

In [ ]:
# Indicies of each class' observations
idx_inactives = np.where( y_train == 0 )[0]
idx_actives   = np.where( y_train == 1 )[0]

# Number of observations in each class
num_inactives = len(idx_inactives)
num_actives   = len(idx_actives)

# Randomly sample from inactives without replacement
np.random.seed(0)
idx_inactives_downsampled = np.random.choice(idx_inactives, size=num_actives, replace=False)

# Join together downsampled inactives with actives
X_train = np.vstack((X_train[idx_inactives_downsampled], X_train[idx_actives]))
y_train = np.hstack((y_train[idx_inactives_downsampled], y_train[idx_actives]))

It is noteworthy that **np.vstack** is used for X_train and **np.hstack** is used for Y_train.  The direction of stacking is different because X_train is a 2-D array and y_train is a 1-D array.

Confirm that the downsampled data set has the correct dimension and active/inactive counts.

In [ ]:
print("# inactives : ", len(y_train) - y_train.sum())
print("# actives   : ", y_train.sum())

In [ ]:
print(len(y_train))
print(len(X_train))
print(len(X_train[0]))

# 8. Build a model using the training set.

Now we are ready to build predictive models using machine learning algorithms available in the scikit-learn library (https://scikit-learn.org/).  This notebook will use Naive Bayes and decisiont tree, because they are relatively fast and simple.

In [ ]:
from sklearn.naive_bayes import BernoulliNB        #-- Naive Bayes
from sklearn.tree import DecisionTreeClassifier    #-- Decision Tree

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

## 8-(1) Naive Bayes

In [ ]:
clf = BernoulliNB()            # set up the NB classification model

In [ ]:
clf.fit( X_train ,y_train )    # Train the model by fitting it to the data.

In [ ]:
y_true, y_pred = y_train, clf.predict( X_train )    # Apply the model to predict the training compound's activity.

In [ ]:
CMat = confusion_matrix( y_true, y_pred )    #-- generate confusion matrix
print(CMat)    # [[TN, FP], 
               #  [FN, TP]]

In [ ]:
acc  = accuracy_score( y_true, y_pred )

sens = CMat[ 1 ][ 1 ] / ( CMat[ 1 ][ 0 ] + CMat[ 1 ][ 1 ] )    # TP / (FN + TP)
spec = CMat[ 0 ][ 0 ] / ( CMat[ 0 ][ 0 ] + CMat[ 0 ][ 1 ] )    # TN / (TN + FP )
bacc = (sens + spec) / 2

y_score = clf.predict_proba( X_train )[:, 1]
auc = roc_auc_score( y_true, y_score )

In [ ]:
print("#-- Accuracy          = ", acc)
print("#-- Balanced Accuracy = ", bacc)
print("#-- Sensitivity       = ", sens)
print("#-- Specificity       = ", spec)
print("#-- AUC-ROC           = ", auc)

When applied to predict the activity of the training compounds, the NB classifier resulted in the accuracy of 0.70 and AUC-ROC of 0.75.  However, the real performance of the model should be evaluated with the test set data, which are not used for model training.

In [ ]:
y_true, y_pred = y_test, clf.predict(X_test)    #-- Apply the model to predict the test set compounds' activity.

In [ ]:
CMat = confusion_matrix( y_true, y_pred )    #-- generate confusion matrix
print(CMat)    # [[TN, FP], 
               #  [FN, TP]]

In [ ]:
acc  = accuracy_score( y_true, y_pred )

sens = CMat[ 1 ][ 1 ] / ( CMat[ 1 ][ 0 ] + CMat[ 1 ][ 1 ] )
spec = CMat[ 0 ][ 0 ] / ( CMat[ 0 ][ 0 ] + CMat[ 0 ][ 1 ] )
bacc = (sens + spec) / 2

y_score = clf.predict_proba( X_test )[:, 1]
auc = roc_auc_score( y_true, y_score )

print("#-- Accuracy          = ", acc)
print("#-- Balanced Accuracy = ", bacc)
print("#-- Sensitivity       = ", sens)
print("#-- Specificity       = ", spec)
print("#-- AUC-ROC           = ", auc)

For the test set, the accuracy is 0.67 and the AUC-ROC is 0.72.  These values are somewhat smaller (by 0.03) than those for the training set.  Also note that the accuracy is no longer the same as the balanced accruacy (which is the average of the sensitivity and specificity).

Some additional performance information may be obtained using **classification_report()**.

In [ ]:
print( classification_report(y_true, y_pred))

## 8-(2) Decision Tree

In [ ]:
clf = DecisionTreeClassifier( random_state=0 )    # set up the DT classification model

In [ ]:
clf.fit( X_train ,y_train )    # Train the model by fitting it to the data (using the default values for all parameters)

In [ ]:
y_true, y_pred = y_train, clf.predict( X_train )    # Apply the model to predict the training compound's activity.

In [ ]:
CMat = confusion_matrix( y_true, y_pred )    #-- generate confusion matrix
print(CMat)    # [[TN, FP], 
               #  [FN, TP]]

In [ ]:
acc  = accuracy_score( y_true, y_pred )

sens = CMat[ 1 ][ 1 ] / ( CMat[ 1 ][ 0 ] + CMat[ 1 ][ 1 ] )    # TP / (FN + TP)
spec = CMat[ 0 ][ 0 ] / ( CMat[ 0 ][ 0 ] + CMat[ 0 ][ 1 ] )    # TN / (TN + FP )
bacc = (sens + spec) / 2

y_score = clf.predict_proba( X_train )[:, 1]
auc = roc_auc_score( y_true, y_score )

In [ ]:
print("#-- Accuracy          = ", acc)
print("#-- Balanced Accuracy = ", bacc)
print("#-- Sensitivity       = ", sens)
print("#-- Specificity       = ", spec)
print("#-- AUC-ROC           = ", auc)

When applied to predict the activity of the **training** compounds, the DT classifier resulted in very high scores (>0.99) for all five performance measures considered here.  However, it does **not** necessarily mean that the model will perform very well for the **test** set compounds.  Let's apply the model to the test set.

In [ ]:
y_true, y_pred = y_test, clf.predict(X_test)    #-- Apply the model to predict the test set compounds' activity.

In [ ]:
CMat = confusion_matrix( y_true, y_pred )    #-- generate confusion matrix
print(CMat)    # [[TN, FP], 
               #  [FN, TP]]

In [ ]:
acc  = accuracy_score( y_true, y_pred )

sens = CMat[ 1 ][ 1 ] / ( CMat[ 1 ][ 0 ] + CMat[ 1 ][ 1 ] )
spec = CMat[ 0 ][ 0 ] / ( CMat[ 0 ][ 0 ] + CMat[ 0 ][ 1 ] )
bacc = (sens + spec) / 2

y_score = clf.predict_proba( X_test )[:, 1]
auc = roc_auc_score( y_true, y_score )

print("#-- Accuracy          = ", acc)
print("#-- Balanced Accuracy = ", bacc)
print("#-- Sensitivity       = ", sens)
print("#-- Specificity       = ", spec)
print("#-- AUC-ROC           = ", auc)

When the DT model was applied to the test set, all performance measures were much worse than those for the training set.  This is a typical example of **outfitting**.

## 9. Model building through cross-validation

In the above section, the models were developed using the default values for many optional hyperparamters, which cannot be learned by the training algorithm.  For example, when building a decision tree model, one should specify how the tree should be deep, how many compounds should be allowed in a single leaf, what is the minimum number of compounds in a single leaf, etc.

The cells below demonstrate how to perform hyperparameter optimization through 10-fold cross-validation.  In this example, five values for each of three hyperparameters used in decision tree are considered (max_depth, min_samples_split, and min_samples_leaf), resulting in a total of 125 combination of the parameter values (=5 x 5 x 5).  For each combination, 10 models are generated (through 10-fold cross validation) and the average performance will be tracked.  The goal is to find the parameter value combination that results in the highest average performance score (e.g., 'roc_auc') from the 10-fold cross validation.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
scores = [ 'roc_auc', 'balanced_accuracy' ]

In [ ]:
ncvs = 10

max_depth_range         = np.linspace( 3, 7, num=5, dtype='int32' )
min_samples_split_range = np.linspace( 3, 7, num=5, dtype='int32' )
min_samples_leaf_range  = np.linspace( 2, 6, num=5, dtype='int32' )

param_grid = dict( max_depth=max_depth_range,
                   min_samples_split=min_samples_split_range,
                   min_samples_leaf=min_samples_leaf_range )

clf = GridSearchCV( DecisionTreeClassifier( random_state=0 ),
                    param_grid=param_grid, cv=ncvs, scoring=scores, refit='roc_auc',
                    return_train_score = True, iid=False)

In [ ]:
clf.fit( X_train, y_train )
print("Best parameter set", clf.best_params_)

If necessary, it is possible to look into the performance data for each parameter value combination (stored in **clf.cv_results_**), as shown in the following cell.

In [ ]:
means_1a = clf.cv_results_['mean_train_roc_auc']
stds_1a  = clf.cv_results_['std_train_roc_auc']

means_1b = clf.cv_results_['mean_test_roc_auc']
stds_1b  = clf.cv_results_['std_test_roc_auc']

means_2a = clf.cv_results_['mean_train_balanced_accuracy']
stds_2a  = clf.cv_results_['std_train_balanced_accuracy']

means_2b = clf.cv_results_['mean_test_balanced_accuracy']
stds_2b  = clf.cv_results_['std_test_balanced_accuracy']

iterobjs = zip( means_1a, stds_1a, means_1b, stds_1b,
                means_2a, stds_2a, means_2b, stds_2b, clf.cv_results_['params'] )

for m1a, s1a, m1b, s1b, m2a, s2a, m2b, s2b, params in iterobjs :

    print( "Grid %r : %0.4f %0.04f %0.4f %0.04f %0.4f %0.04f %0.4f %0.04f"
           % ( params, m1a, s1a, m1b, s1b, m2a, s2a, m2b, s2b))

Uncomment the following cell to look into additional performance data stored in cv_result_.

In [ ]:
#print(clf.cv_result_)

It is important to understand that each model built through 10-fold cross-validation during hyperparameter optimization uses only 90% of the compounds in the training set and the remaining 10% is used for testing that model.  After all parameter value combinations are evaluated, the best parameter values are selected and used to rebuild a model from **all** compounds in the training set.  **GridSearchCV()** takes care of this last step automatically.  Therefore, there is no need to take an extra step to build a model using **cls.fit()** after hyperparameter optimization.

In [ ]:
y_true, y_pred = y_train, clf.predict( X_train )    # Apply the model to predict the training compound's activity.

In [ ]:
CMat = confusion_matrix( y_true, y_pred )    #-- generate confusion matrix
print(CMat)    # [[TN, FP], 
               #  [FN, TP]]

In [ ]:
acc  = accuracy_score( y_true, y_pred )

sens = CMat[ 1 ][ 1 ] / ( CMat[ 1 ][ 0 ] + CMat[ 1 ][ 1 ] )    # TP / (FN + TP)
spec = CMat[ 0 ][ 0 ] / ( CMat[ 0 ][ 0 ] + CMat[ 0 ][ 1 ] )    # TN / (TN + FP )
bacc = (sens + spec) / 2

y_score = clf.predict_proba( X_train )[:, 1]
auc = roc_auc_score( y_true, y_score )

In [ ]:
print("#-- Accuracy          = ", acc)
print("#-- Balanced Accuracy = ", bacc)
print("#-- Sensitivity       = ", sens)
print("#-- Specificity       = ", spec)
print("#-- AUC-ROC           = ", auc)

Compare these performance data with those from section 8-(2) (for the training set).  When the default values were used, the DT model gave >0.99 for all performance measures, but the current models (developed using hyperparameter optimization) have much lower values, ranging from 0.73 to 0.83.  Again, however, what really matters is the performance against the test set, which contains the data not used for model training. 

In [ ]:
y_true, y_pred = y_test, clf.predict(X_test)    #-- Apply the model to predict the test set compounds' activity.

In [ ]:
CMat = confusion_matrix( y_true, y_pred )    #-- generate confusion matrix
print(CMat)    # [[TN, FP], 
               #  [FN, TP]]

In [ ]:
acc  = accuracy_score( y_true, y_pred )

sens = CMat[ 1 ][ 1 ] / ( CMat[ 1 ][ 0 ] + CMat[ 1 ][ 1 ] )
spec = CMat[ 0 ][ 0 ] / ( CMat[ 0 ][ 0 ] + CMat[ 0 ][ 1 ] )
bacc = (sens + spec) / 2

y_score = clf.predict_proba( X_test )[:, 1]
auc = roc_auc_score( y_true, y_score )

print("#-- Accuracy          = ", acc)
print("#-- Balanced Accuracy = ", bacc)
print("#-- Sensitivity       = ", sens)
print("#-- Specificity       = ", spec)
print("#-- AUC-ROC           = ", auc)

Now we can see that the model from hyperparameter optimization gives better performance data against the test set, compared to the model developed using the default parameter values.  Importantly, the model from hyperparameter optimization shows smaller differences in performance measures between the training and test sets, indicatiing that the issue of outffiting has been alleviated substantially.

# Exercises

In this assignment, we will build predictive models using the same aromatase data.

**step 1** Show the following information to make sure that the activity data in the **df_activity** data frame is still available.

- The first five lines of **df_activity**

In [ ]:
# Write your code in this cell.


- The counts of active/inactive compounds in **df_activity**

In [ ]:
# Write your code in this cell.


**Step 2** Show the following information to make sure the structure data is still available.

- The first five lines of **df_smiles**

In [ ]:
# Write your code in this cell.


- the number of rows of **df_smiles**

In [ ]:
# Write your code in this cell.


**Step 3** Generate the (ECFP-equivalent) circular fingerprints from the SMILES strings.
- Use RDKit to generate 1024-bit-long circular fingerprints.
- Set the radius of the circular fingerprint to 2.
- Store the fingerprints in a data_frame called **df_fps** (along with the CIDs).
- Print the dimension of **df_fps**.
- Show the first five lines of **df_fps**.

In [ ]:
# Write your code in this cell


**Step 4** Merge the **df_activity** and **df_fps** data frames into a data frame called **df_data**
- Join the two data frames using the CID column as keys.
- Remove the rows that have any NULL values (i.e., compounds for which the fingerprints couldn't be generated).
- Print the dimension of **df_data**.
- Show the first five lines of **df_data**.

In [ ]:
# Write your code in this cell.



**Step 5** Prepare input and output data for model building
- Load the fingerprint data into 2-D array (X) and the activity data into 1-D array (y).
- Show the dimension of X and y.

In [ ]:
# Write your code in this cell.



- Remove zero-variance features from X (if any).

In [ ]:
# Write your code in this cell.



- Split the data set into training and test sets (90% vs 10%) (using random_state=3100).
- Print the dimension of X and y for the training and test sets.

In [ ]:
# Write your code in this cell.



- Balance the training data set through downsampling.
- Show the number of inactive/active compounds in the downsampled training set.

In [ ]:
# Write your code in this cell.


**Step 6** Building a Random Forest model using the balanced training data set.
- First read the followng documents about random forest (https://scikit-learn.org/stable/modules/ensemble.html#forest and https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier). 
- Use 10-fold cross validation to select the best value for the "n_estimators" parameter that maximizes the **balanced accuracy**.  Test 40 values from 5 to 200 with an increment of 5 (e.g., 5, 10, 15, 20, ..., 190, 195, 200).
- For parameters 'max_depth', 'min_samples_leaf', and 'min_samples_split', use the best values found in Section 9.
- For other parameters, use the default values.
- For each parameter value, print the mean balanced accuracies (for both training and test from cross validation). 

In [ ]:
# Write your code in this cell.



**Step 7** Apply the developed RF model to predict the activity of the **training** set compounds.

- Report the confusion matrix.
- Report the accuracy, balanced accurayc, sensitivity, specificity, and auc-roc.

In [ ]:
# Write your code in this cell.


**Step 8** Apply the developed RF model to predict the activity of the **test** set compounds.

- Report the accuracy, balanced accurayc, sensitivity, specificity, and auc-roc.

In [ ]:
# Write your code in this cell.


**Step 9** Read a recent paper published in *Chem. Res. Toxicol.* (https://doi.org/10.1021/acs.chemrestox.7b00037) and answer the following questions (in no more than five sentences for each question).

- What different approaches did the paper take to develop prediction models (compared to those used in this notebook)?
- How different are the models reported in the paper from those constructed in this paper (in terms of the performance measures)? 
- What would you do to develop models with improved performance?

Write your answers in this cell.

- 

- 

- 